In [1]:
import cv2
import cv2.aruco as aruco
import numpy as np
import gym
import math
import vision_arena
import pybullet as p
import time

# Detecting and cropping the image

In [2]:
def Image(is_first = False):
    global A1, A2, env
    if is_first:
        print("Instructions:")
        print("Crop The Image To Arena Size (Removing Additional Black Stripes).")
        img = env.camera_feed()
        A1 = np.array(img.shape[:2])
        
        r = cv2.selectROI(img)
        crop = img[int(r[1]):int(r[1]+r[3]),int(r[0]):int(r[0]+r[2])]
        
        A2 = np.array(crop.shape[:2])
        A1 = (A1 - A2) / 2
        
        cv2.destroyAllWindows()
    else:
        img = env.camera_feed()
        return img

# Required postion and vectors of bot

In [3]:
def Bot_Pos():
    global A1, A2, Last, aruco_dict
    while True:
        img = Image()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        corners, ids, _ = aruco.detectMarkers(gray, aruco_dict, parameters=aruco.DetectorParameters_create())
        try:
            for i in range(len(ids)):
                id = ids[i][0]
                if id == 107:
                    L = corners[i][0]
                    X = int((L[0][0]+L[2][0])/2)
                    Y = int((L[0][1]+L[2][1])/2)

                    MX = int((Y-A1[1])/(A2[1]/9))
                    MY = int((X-A1[0])/(A2[0]/9))

                    X1 = int(((L[0][0]+L[1][0])/2))
                    Y1 = int(((L[0][1]+L[1][1])/2))
                    X2 = int(((L[2][0]+L[3][0])/2))
                    Y2 = int(((L[2][1]+L[3][1])/2))
                    return (X1,Y1),(X2,Y2),(X,Y),(MX,MY)
        except:
            if Last in [[0,0],[0,8],[8,8],[8,0]]:
                Reverse()
            else:
                Forward()

# Decides the end postion according to start position

In [4]:
def stop(start):
    if start==(4,0):
        return (4,3)
    elif start==(4,8):
        return (4,5)
    elif start==(0,4):
        return (3,4)
    else:
        return (5,4)

# Array of arena using opencv color detection

In [5]:
def Detection():
    global A, A1, A2, A3
    Dict = {0:"Red",1:"Yellow"}
    List = [[1,2,3],[4,5,6]]

    print("Take Double Color Sample For Both Colours (Red, Yellow) One By One.")
    thresh = 15

    for i in range(2):
        print("Select",Dict[i],"Color",end = "\t")

        img = Image()

        r1 = cv2.selectROI(img)
        crop1 = img[int(r1[1]):int(r1[1]+r1[3]),int(r1[0]):int(r1[0]+r1[2])]

        lower1 = np.array([crop1[:,:,0].min()-thresh,crop1[:,:,1].min()-thresh,crop1[:,:,2].min()-thresh])
        upper1 = np.array([crop1[:,:,0].max()+thresh,crop1[:,:,1].max()+thresh,crop1[:,:,2].max()+thresh])

        r2 = cv2.selectROI(img)
        crop2 = img[int(r2[1]):int(r2[1]+r2[3]),int(r2[0]):int(r2[0]+r2[2])]

        lower2 = np.array([crop2[:,:,0].min()-thresh,crop2[:,:,1].min()-thresh,crop2[:,:,2].min()-thresh])
        upper2 = np.array([crop2[:,:,0].max()+thresh,crop2[:,:,1].max()+thresh,crop2[:,:,2].max()+thresh])

        print(Dict[i],"Color Selected Sucessfully.")

        blur = cv2.bilateralFilter(img,9,75,75) 
        mask = cv2.bitwise_or(cv2.inRange(blur, lower1, upper1), cv2.inRange(blur, lower2, upper2))
        mask = cv2.dilate(mask, np.ones((5,5), dtype = np.uint8), iterations = 1)
        mask = cv2.erode(mask, np.ones((3,3), dtype = np.uint8), iterations = 1)

        res = cv2.bitwise_and(img, img, mask = mask)

        cv2.imshow("Detection",res)
        print("Press Any Key To Continue.")
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        test = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        _, test = cv2.threshold(test, 5, 95, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(test, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

        for contour in contours:
            area = cv2.contourArea(contour)
            if area>100:
                rect = cv2.minAreaRect(contour)
                a=rect[1][0]*rect[1][1]
                m = cv2.moments(contour)
                Y = (m["m10"]/m["m00"])
                X = (m["m01"]/m["m00"])

                y = int((Y-A1[0])/(A2[0]/9))
                x = int((X-A1[1])/(A2[1]/9))

                A3[y][x][0] = X
                A3[y][x][1] = Y

                if (area/a) > 0.9:
                    A[x][y] = List[i][0]
                elif (area/a) <= 0.9 and (area/a) > 0.77:
                    A[x][y] = List[i][1]
                else:
                    A[x][y] = List[i][2]

        print("Shape Detection For Color",Dict[i],"Done Sucessfully.")

    A3[4][4][0] = A2[0]/2+A1[0]
    A3[4][4][1] = A2[1]/2+A1[1]

    A3[4][0][0] = A3[0][3][1]
    A3[4][0][1] = A3[1][4][0]

    A3[8][4][0] = A3[4][7][1]
    A3[8][4][1] = A3[3][8][0]

    A3[4][8][0] = A3[8][3][1]
    A3[4][8][1] = A3[7][4][0]

    A3[0][4][0] = A3[4][1][1]
    A3[0][4][1] = A3[3][0][0]

    cv2.destroyAllWindows()

# Returns the correct neighbour of current position 

In [6]:
def neighbours(t,rot):
    lst=[]
    i=t[0]
    j=t[1]
#     if j==0 :
       
#         if i==4 or i==0 :
#             lst.append((i,j+1))
        
#         if i!=0:
#             lst.append((i-1,j)
    if j==0:
        if i==4  or i==0:
            lst.append((i,j+1))              
        if i!=0 :
            if i!=4:
                lst.append((i-1,j))
            elif i==4 and rot==False:
                lst.append((i-1,j))
    if j==8 :
        if i==4 or i==8:
            lst.append((i,j-1))
        if i!=8:
            if i!=4:
                lst.append((i+1,j))
            elif i==4 and rot==False:
                lst.append((i+1,j))
            

    if j in range(1,8) and i<4:
        if i==0:
            if j==4 :
                if rot==False:
                    lst.append((i,j+1))
                lst.append((i+1,j))
            if j!=4:
                lst.append((i,j+1))
        if i==1:
            lst.append((i+1,j))
            lst.append((i-1,j))
        if i==2:
            if j==4 :
                if rot==True:
                    lst.append((i+1,j))
                if rot==False:
                    lst.append((i,j+1))
                lst.append((i-1,j))
                
            if j==6:
                lst.append((i+1,j))
            elif j!=4:
                lst.append((i,j+1))
        if i==3:
            if j==2:
                lst.append((i-1,j))
            if j==6:
                lst.append((i+1,j))
                
            
            
    elif j in range(1,8) and i>4:
        if i==8:
            
            if j==4 :
                if rot==False:
                    lst.append((i,j-1))
                lst.append((i-1,j))
            
            if j!=4:
                
                lst.append((i,j-1))
            
            
#             if j==4 :
#                 lst.append((i-1,j))
#             lst.append((i,j-1))
        if i==7:
            lst.append((i-1,j))
            lst.append((i+1,j))
        if i==6:
            if j==4 :
                if rot==True:
                    lst.append((i-1,j))
                if rot==False:
                    lst.append((i,j-1))
                lst.append((i+1,j))
               
#             if j==4 :
#                 lst.append((i,j-1))
#                 lst.append((i+1,j))
            if j==2:
                lst.append((i-1,j))
            elif j!=4:
                lst.append((i,j-1))
        if i==5:
            if j==2 :
                lst.append((i-1,j))
            if j==6:
                lst.append((i+1,j))
    if i==4 and j not in[0,8] :
        if j ==7:
            
            lst.append((i,j-1))
            lst.append((i,j+1))
        elif j ==1:
            
            lst.append((i,j+1))
            lst.append((i,j-1))
        elif j == 6:
            if rot==False:
                lst.append((i+1,j))
            if rot==True:
                lst.append((i,j-1))
            lst.append((i,j+1))
#             lst.append((i+1,j))
#             lst.append((i,j+1))
        elif j == 2:
            if rot==False:
                lst.append((i-1,j))
            if rot==True:
                lst.append((i,j+1))
            lst.append((i,j-1))
            
    if i==4 and ( j==3 or j==5):
        lst.append((4,4))
    if(i==4 and j==4):
        return []
        
        
    
        
    return lst
neighbours((4,6),True)

[(4, 5), (4, 7)]

In [7]:
def nodes(t):
    return A[t[0]][t[1]]
    
            

In [8]:
def visited_in_path(path,node):
    if node in path:
        return False
    return True

# End position according to start 

In [9]:
def End_list(start):
    x=start[0]
    y=start[1]
    End=[]
    if x==0 and y==4:
        End.append((0,3))
        End.append((2,3))
    elif x==4 and y==0:
        End.append((5,0))
        End.append((5,2))
    elif x==8 and y==4:
        End.append((8,5))
        End.append((6,5))
    elif x==4 and y==8:
        End.append((3,8))
        End.append((3,6))
    return End
                

# End Row according to start

In [10]:
def End_row(start):
    x=start[0]
    y=start[1]
    End=[]
    if x==0 and y==4:
        End.append((0,3))
        End.append((2,3))
        End.append((0,4))
        End.append((1,4))
        End.append((2,4))
    elif x==4 and y==0:
        End.append((5,0))
        End.append((5,2))
        End.append((4,0))
        End.append((4,1))
        End.append((4,2))
    elif x==8 and y==4:
        End.append((8,5))
        End.append((6,5))
        End.append((6,4))
        End.append((7,4))
        End.append((8,4))
    elif x==4 and y==8:
        End.append((3,8))
        End.append((3,6))
        End.append((4,6))
        End.append((4,7))
        End.append((4,8))
    return End

In [11]:
def init(is_first = False):
    global Interpretation, A, A1, A2, A3, env, aruco_dict, Start, End, Last
    
    if is_first:
        Interpretation = {"Black":0, "SR":1, "CR":2, "TR":3, "SY":4, "CY":5, "TY":6}
        A = np.zeros([9,9], dtype = np.int32) # Arena
        A1 = np.zeros([2], dtype = np.int32) # Black Stripes Thickness
        A2 = np.zeros([2], dtype = np.int32) # Image Size
        A3 = np.zeros([9,9,2], dtype = np.int32) # Node's Co-Orinates in Pixel Form

        env = gym.make("vision_arena-v0")
        aruco_dict = aruco.Dictionary_get(aruco.DICT_ARUCO_ORIGINAL)

        Image(True)
        Detection()
    else:
        env.remove_car()
        env.respawn_car()
        _ = env.camera_feed()
        
    _, _, _, (X,Y) = Bot_Pos()
    
    Start = [X,Y]
    Last = [X,Y]

    if [X,Y] == [0,4]:
        End = [[X,Y-1],[X+2,Y-1]]
    elif [X,Y] == [4,8]:
        End = [[X-1,Y],[X-1,Y-2]]
    elif [X,Y] == [8,4]:
        End = [[X,Y+1],[X-2,Y+1]]
    elif [X,Y] == [4,0]:
        End = [[X+1,Y],[X+1,Y+2]]

    
    print("Start",Start)

In [12]:
init(True)
A[4][0]=A[0][4]=A[8][4]=A[4][8]=7
A[4][4]=8
_, _, _,(X,Y) = Bot_Pos()
start=(X,Y)
sx=start[0]
sy=start[1]
st=stop(start)
End=End_list(start)
Endrow=End_row(start)
rot=False

A

Instructions:
Crop The Image To Arena Size (Removing Additional Black Stripes).
Take Double Color Sample For Both Colours (Red, Yellow) One By One.
Select Red Color	Red Color Selected Sucessfully.
Press Any Key To Continue.
Shape Detection For Color Red Done Sucessfully.
Select Yellow Color	Yellow Color Selected Sucessfully.
Press Any Key To Continue.
Shape Detection For Color Yellow Done Sucessfully.
Start [8, 4]


array([[5, 5, 4, 2, 7, 2, 1, 3, 4],
       [4, 0, 0, 0, 6, 0, 0, 0, 4],
       [3, 0, 5, 4, 1, 1, 6, 0, 2],
       [3, 0, 4, 0, 3, 0, 5, 0, 1],
       [7, 1, 5, 4, 8, 6, 1, 2, 7],
       [3, 0, 1, 0, 6, 0, 3, 0, 6],
       [4, 0, 2, 4, 6, 3, 2, 0, 2],
       [4, 0, 0, 0, 1, 0, 0, 0, 3],
       [6, 1, 4, 5, 7, 6, 5, 1, 1]])

# Algo for shortest path from intial to final position

In [13]:
def bfs(s,d):
    result=list(list())
    q=list(list())
    path=list()
    path.append(s)
    q.append(path)
    global rot
    k=0
    while len(q)!=0:
        path=q[0]
        q.pop(0)
        last=path[-1]
        if last in End:
            rot=True
        if nodes(last)==d and len(path)>1:
            result.append(path)
            k+=1
            if k==1:
                return result
            
        nbrs=neighbours(last,rot)
        
        for nbr in nbrs:
            if(visited_in_path(path,nbr) and nodes(nbr)!=0):
                newpath=[]
                newpath.extend(path)
                newpath.append(nbr)
                q.append(newpath)
                
        

# Movement Functions

In [14]:
def Distance(a1,b1,a2,b2):
    return ((b2-b1)**2 + (a2-a1)**2)**(1/2)

In [15]:
def Angle(a1,b1,a2,b2):
    bot_vector = complex(a1,b1)
    path_vector = complex(a2,b2)
    angle = np.angle(path_vector/bot_vector)
    angle = (angle*180)/3.14
    return angle

In [16]:
def Forward(d=20):
    global env
    speed = min(10,max(d-12,5))
    env.move_husky(speed,speed,speed,speed)
    for _ in range(min(5,d-11)):
        p.stepSimulation()
    env.move_husky(0,0,0,0)
    p.stepSimulation()

In [17]:
def Reverse(d=20):
    global env
    speed = min(10,max(d-12,5))
    env.move_husky(-speed,-speed,-speed,-speed)
    for _ in range(min(5,d-11)):
        p.stepSimulation()
    env.move_husky(0,0,0,0)
    p.stepSimulation()

In [18]:
def Left(theta):
    global env
    speed = min(17,theta+2)
    env.move_husky(-speed,speed,-speed,speed)
    for _ in range(5):
        p.stepSimulation()
    env.move_husky(0,0,0,0)
    p.stepSimulation()

In [19]:
def Right(theta):
    global env
    speed = min(17,theta+2)
    env.move_husky(speed,-speed,speed,-speed)
    for _ in range(5):
        p.stepSimulation()
    env.move_husky(0,0,0,0)
    p.stepSimulation()

In [20]:
def Move(Best_Path):
    global A3, Last, env
    for i,j in Best_Path:
        X,Y = A3[i][j]
        while True:
            (X1,Y1), (X2,Y2), (MX,MY), (x,y) = Bot_Pos()
            Last = [x,y]
            d = Distance(X,Y,MX,MY)
            if d > 12:
                theta = Angle((X1-X2),(Y1-Y2),(X - MX),(Y - MY))
                img = Image()
                if (theta <= 5 and theta >=-5):
                    Forward(int(d))
                elif d < 24 and theta >= 172 and theta <= -172:
                     break
                elif theta >= 130 or theta <= -130:
                    if theta >= 175 or theta <= -175:
                        Reverse(int(d))
                    elif theta >= 130:
                        Left(int(180-theta))
                    elif theta <= -130:
                        Right(int(180-abs(theta)))
                elif theta < 0:
                    Left(-int(theta))
                elif theta > 0:
                    Right(int(theta))
            else:
                break

In [21]:
bot=start

In [22]:
def Run():
    global Interpretation,env,bot,rot
    while bot!=st:
        d=Interpretation[env.roll_dice()]
        result=bfs(bot,d)
        print(d)
        print(result)
        if result!=None:
            Move(result[0][1:])
        if  result!=None:
            bot=result[-1][-1]
        if result==None and bot not in Endrow:
            rot=False
    
        
        
        

In [23]:
if __name__ == "__main__":
    Run()

4
[[(8, 4), (8, 3), (8, 2)]]
3
[[(8, 2), (8, 1), (8, 0), (7, 0), (6, 0), (5, 0)]]
2
[[(5, 0), (4, 0), (3, 0), (2, 0), (1, 0), (0, 0), (0, 1), (0, 2), (0, 3)]]
2
[[(0, 3), (0, 4), (0, 5)]]
1
[[(0, 5), (0, 6)]]
1
[[(0, 6), (0, 7), (0, 8), (1, 8), (2, 8), (3, 8)]]
1
[[(3, 8), (4, 8), (4, 7), (4, 6)]]
1
[[(4, 6), (5, 6), (6, 6), (6, 5), (6, 4), (7, 4)]]
6
[[(7, 4), (6, 4)]]
1
[[(6, 4), (7, 4)]]
3
None
6
[[(7, 4), (6, 4)]]
6
[[(6, 4), (5, 4)]]


In [24]:
A

array([[5, 5, 4, 2, 7, 2, 1, 3, 4],
       [4, 0, 0, 0, 6, 0, 0, 0, 4],
       [3, 0, 5, 4, 1, 1, 6, 0, 2],
       [3, 0, 4, 0, 3, 0, 5, 0, 1],
       [7, 1, 5, 4, 8, 6, 1, 2, 7],
       [3, 0, 1, 0, 6, 0, 3, 0, 6],
       [4, 0, 2, 4, 6, 3, 2, 0, 2],
       [4, 0, 0, 0, 1, 0, 0, 0, 3],
       [6, 1, 4, 5, 7, 6, 5, 1, 1]])